In [516]:
# https://www.sktime.net/en/v0.24.1/examples/transformation/rocket.html

from sktime.transformations.panel.rocket import Rocket
from matplotlib import pyplot as plt
from sktime.utils.plotting import plot_series
from sklearn.linear_model import RidgeClassifierCV
import ipywidgets as widgets
import numpy as np
import pandas as pd
import os
from sktime.pipeline import make_pipeline

In [517]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [518]:
#num_ops = 9

In [519]:
# Need multiple rocket kernels - an array propertional to the number of operation time series
#trf = Rocket(num_kernels=512)
#classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10))
#train_ts_combined = None
#train_metrics = load_train_metrics()

In [520]:
# Seperate set of training data per column in rocket_tfs

In [521]:
# test_num has to be zero based
def import_single_test_csv_into_train_data(file, test_num, train_test_count, train_data):
    df = pd.read_csv(file)
    num_ops = len(df.columns)
    op_num = 0
    for op_name, op_series in df.items():
        time_len = len(op_series)
        # Need to add this to the training_data
        #add_to_train_data(op_name, op_series, test_num, train_data)
        train_data[op_num] = np.zeros((time_len, train_test_count),dtype=float)
        train_data[op_num][:, test_num] = op_series
        op_num += 1
    return train_data

In [522]:
#train_data = {}
# Import test
#trf = Rocket(num_kernels=512)
#import_single_test_csv_into_train_data(trf, "/tmp/testCSV/Test_001_18_01_2024_05_28_25.csv", 1, 32, train_data)
#train_data['dosAttackTrigger'].shape

In [523]:
# Trains a ROCKET transformer for the numbered series
def train_rocket_for_op_num(train_data, train_ts_combined, rocket_trfs, op_num):
    rocket_trfs[op_num] = Rocket(num_kernels=512)
    trf = rocket_trfs[op_num]
    trf.fit(train_data[op_num])
    fitted = trf.transform(train_data[op_num])
    print("fitted=",fitted)
    #print("train_ts_combined=" + str(train_ts_combined))
    if train_ts_combined is None:
        train_ts_combined = fitted
    else:
        train_ts_combined.join(fitted, lsuffix=("series" + str(op_num)))
    return train_ts_combined

In [524]:
def find_data_in_dir(csv_directory_train, csv_directory_test, metrics_dir, train_split_num):
    files_train = os.listdir(csv_directory_train)
    files_test = os.listdir(csv_directory_test)
    metrics_train = pd.read_csv("/tmp/testCSV/metrics-train.csv")
    metrics_test = pd.read_csv("/tmp/testCSV/metrics-test.csv")
    return files_train, metrics_train, files_test, metrics_test        

In [525]:
def rocket_complete(csv_directory_train, csv_directory_test, train_split_num, num_ops):
    train_filenames, train_metrics, test_filenames, test_metrics = find_data_in_dir(csv_directory_train, csv_directory_test, "", ~train_split_num)
    # Load the CSV data for a single Test
    
    train_test_count = len(train_filenames)
    train_data = {}
    rocket_trfs = {}
    train_ts_combined = None
    classifier = RidgeClassifierCV(alphas = np.logspace(-3, 3, 10))
    train_num = 0
    for file in train_filenames:
        file_in_dir = csv_directory_train + "/" + file
        # Split it into columns to extract each time series
        train_data = import_single_test_csv_into_train_data(file_in_dir, train_num, train_test_count, train_data)
        train_num += 1
        
    print("train_data[0]=",train_data[0].shape)
    return train_data
        
    for op_num in range(0,num_ops):
        # Each column has to be processed to create a feature
        # Then need to unify the features from all columns to provide a common set
        train_ts_combined = train_rocket_for_op_num(train_data, train_ts_combined, rocket_trfs, op_num)
        print("train_ts_combined size = " + str(train_ts_combined))
        
    
    # Fit classifier up to a fixed limit
    # TODO: need to use regression, not classifier?
    classifier.fit(train_ts_combined, train_metrics)
    # Then test the prediction on the previous ones
    
    classifier.score(test_x_transform, test_y)
    

In [526]:
dir = "/tmp/testCSV"
split_num = 20
num_ops = 4
td = rocket_complete("/tmp/testCSV/train/", "/tmp/testCSV/test/", split_num, num_ops)
td

train_data[0]= (300, 20)


{0: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 1: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 2: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 3: array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.,

In [559]:
import pandas as pd

cols = ["instances", "time points"] + [f"var_{i}" for i in range(2)]
X = pd.concat(
    [
        pd.DataFrame([[0, 0, 1, 4], [0, 1, 2, 5], [0, 2, 3, 6]], columns=cols),
        pd.DataFrame([[1, 0, 1, 4], [1, 1, 2, 55], [1, 2, 3, 6]], columns=cols),
        pd.DataFrame([[2, 0, 1, 42], [2, 1, 2, 5], [2, 2, 3, 6]], columns=cols),
    ]
).set_index(["instances", "time points"])

In [534]:
X

var_0  var_1
instances time points              
0         0                1      4
          1                2      5
          2                3      6
1         0                1      4
          1                2     55
          2                3      6
2         0                1     42
          1                2      5
          2                3      6

In [594]:
from sktime.datatypes import check_raise, convert_to

df_series1 = pd.read_csv("/tmp/testCSV/train/Test_001_18_01_2024_05_28_25.csv")
# Need to ensure it has zeros for all columns if not present!
df_series2 = pd.read_csv("/tmp/testCSV/train/Test_002_18_01_2024_05_28_25.csv")
zzz = [df_series1, df_series2]
check_raise(zzz, mtype="df-list")

True

In [595]:
convert_to(zzz, to_type="pd-multiindex")

# Load all the dataframes for either test or training
# Ensure all their missing columns can be filled in with zeros
# Make a list of them, use check_raise and the df-list mtype to ensure they are valid
# Pass to ROCKET pipeline as seen

dosAttackTrigger  distortBaseScanPMB2  \
instances timepoints                                          
0         0                        0.0                  0.0   
          1                        0.0                  0.0   
          2                        0.0                  0.0   
          3                        0.0                  0.0   
          4                        0.0                  0.0   
...                                ...                  ...   
1         295                      0.0                  NaN   
          296                      0.0                  NaN   
          297                      0.0                  NaN   
          298                      0.0                  NaN   
          299                      0.0                  NaN   

                      delayBaseScanOmni  delayBaseScanPMB2  
instances timepoints                                        
0         0                         0.0                0.0  
          1                         0.0                0.0  
          2                         0.0                0.0  
          3                         0.0                0.0  
          4                         0.0                0.0  
...                                 ...                ...  
1         295                       NaN                NaN  
          296                       NaN                NaN  
          297                       NaN                NaN  
          298                       NaN                NaN  
          299                       NaN                NaN  

[600 rows x 4 columns]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [583]:
cols = ["instances", "time points", "dosAttackTrigger", "distortBaseScanPMB2", "delayBaseScanOmni", "delayBaseScanPMB2"]
# Need time series of lengths - 0 to 300
timesteps = np.arange(0,300)
df["timesteps"] = timesteps
df


,dosAttackTrigger,timesteps
distortBaseScanPMB2,,
0.0,0.0,0
0.0,0.0,1
0.0,0.0,2
0.0,0.0,3
0.0,0.0,4
...,...,...
0.0,0.0,295
0.0,0.0,296
0.0,0.0,297


In [555]:
cols = ["instances", "time points"] + [f"var_{i}" for i in range(2)]
X = pd.concat(
    [df_series1["dosAttackTrigger"], df_series2["dosAttackTrigger"]], columns=cols).set_index(["instances", "time points"])

SyntaxError: incomplete input (1611151946.py, line 3)

In [585]:
get_examples(mtype="df-list", as_scitype="Panel")[0]

NameError: name 'get_examples' is not defined

In [598]:
import numpy as np
x = np.array([1, 2, 3, 4, 5])
f = lambda x: x ** 2
squares = f(x)
squares

array([ 1,  4,  9, 16, 25])